In [ ]:
from Utils.my_utils import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative
from Recommenders.NonPersonalizedRecommender import *
import os
from random import randint, seed

In [ ]:
# setta anche il seed di random per sicurezza
seed = randint(1,1000)
print("Seed: " +str(seed))
np.random.seed(seed)
users, items, users_to_eval = readData()
URM = newPreProcess()
URM_train_full, URM_train, URM_test, URM_validation = splitURM(URM)

In [ ]:
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [ ]:
output_folder_path = "result_experiments/"
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

saved_folder_path = "result_best_models/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
    
recommender_class = SLIMElasticNetRecommender    
n_cases = 15
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [ ]:
runHyperparameterSearch_Collaborative(recommender_class,
       URM_train = URM_train,
       URM_train_last_test = URM_train_full,
       metric_to_optimize = metric_to_optimize,
       cutoff_to_optimize = cutoff_to_optimize,
       evaluator_validation_earlystopping = evaluator_validation,
       evaluator_validation = evaluator_validation,
       evaluator_test = evaluator_test,
       output_folder_path = output_folder_path,
       parallelizeKNN = True,
       allow_weighting = True,
       resume_from_saved = True,
       save_model = "best",
       similarity_type_list = ["tversky"],
       n_cases = n_cases,
       n_random_starts = n_random_starts)

In [ ]:
# load the model with all best parameters
recommender_object = SLIMElasticNetRecommender(URM_train_full)
recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_best_model_last.zip")
#recommender_object.load_model(output_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_best_model.zip")

In [ ]:
# evaluate recommender
evaluate_algorithm(URM_test, recommender_object, at=10)

In [ ]:
# recommend items to users
recommendations = recommender_object.recommend(users_to_eval, cutoff=10)

# store recommendations
newStoreRecommendations(users_to_eval, recommendations)

In [ ]:
# save the model with all best parameters overwriting the old one
"""
saved_folder_path = "result_best_models/" + str(seed) + "/"
if not os.path.exists(saved_folder_path):
    os.makedirs(saved_folder_path)
recommender_object.save_model(saved_folder_path, file_name = recommender_object.RECOMMENDER_NAME + "_my_own_save.zip")
"""